In [73]:
import pandas as pd
import numpy as np

#Importacion xgboost
import xgboost as xgb

#Importacion preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#Importacion metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

#Librerias para visualizacion
import seaborn as sb
import matplotlib.pyplot as plt

plt.style.use('default')

In [74]:
def getData():
  return pd.read_csv("https://docs.google.com/spreadsheets/d/1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0/export?format=csv&gid=2054774482")

In [75]:
df = getData()
df.head()

,anios_estudiados,barrio,categoria_de_trabajo,edad,educacion_alcanzada,estado_marital,ganancia_perdida_declarada_bolsa_argentina,genero,horas_trabajo_registradas,religion,rol_familiar_registrado,tiene_alto_valor_adquisitivo,trabajo
0,17,Palermo,empleado_provincial,39,universidad_3_anio,sin_matrimonio,2174,hombre,40,cristianismo,sin_familia,0,entretenimiento
1,17,Palermo,monotibutista,50,universidad_3_anio,matrimonio_civil,0,hombre,13,cristianismo,casado,0,directivo_gerente
2,13,Palermo,relacion_de_dependencia,38,4_anio,divorciado,0,hombre,40,cristianismo,sin_familia,0,limpiador
3,11,Palermo,relacion_de_dependencia,53,2_anio,matrimonio_civil,0,hombre,40,judaismo,casado,0,limpiador
4,17,Balvanera,relacion_de_dependencia,28,universidad_3_anio,matrimonio_civil,0,mujer,40,judaismo,casada,0,profesional_especializado


In [76]:
#Funcion que devuelve un 1 si esa row posee esta combinacion en especifico de valores en sus columnas "trabajo" y "rol_familiar_registrado"
def set_value_row_casado_trabajo(row):
    if (row.rol_familiar_registrado == "casado" and (row.trabajo == "profesional_especializado" or row.trabajo == "directivo_gerente" )):
      return 1                 
    else:
      return 0

def feature_engineering_xg(df):

    #Creo columna opera_en_bolsa que indica si opero en bolsa o no
    df["opera_en_bolsa"] = df["ganancia_perdida_declarada_bolsa_argentina"].apply(lambda x: 1 if x != 0 else 0)

    #Creo columna es_hombre que indica si la persona es hombre o no
    df["es_hombre"] = df["genero"].apply(lambda x: 1 if x == "hombre" else 0)

    #Unifico los labels "casado" y "casada" en "casado"
    df.rol_familiar_registrado.replace(to_replace=["casada"],  value=["casado"], inplace=True)

    #Creo columna casado_trabajo que indica si ese usuario posee una combinacion especifica entre la columna rol_familiar_registrado y trabajo
    df["casado_trabajo"] = df.apply(lambda row: set_value_row_casado_trabajo(row), axis= 1)

    #Dropeo columnas que no considero necesarias para la prediccion
    df.drop(columns = ["edad", "horas_trabajo_registradas", "barrio", "genero", "ganancia_perdida_declarada_bolsa_argentina", "anios_estudiados"], inplace=True)

    #Aplico one hot enconding a estas columnas en especifico
    df = pd.get_dummies(df, drop_first=True, columns=["categoria_de_trabajo", "educacion_alcanzada", "estado_marital", "religion", "rol_familiar_registrado", "trabajo"])

    #Borro una columna de cada uno de los one hot creados
    df.drop(columns = ["categoria_de_trabajo_empleado_municipal", "educacion_alcanzada_1_anio", "estado_marital_matrimonio_civil", "religion_budismo", "rol_familiar_registrado_con_hijos", "trabajo_ejercito"], inplace=True)

    if "tiene_alto_valor_adquisitivo" in df.columns:
      label_encoder = preprocessing.LabelEncoder()
      label_encoder.fit(df.tiene_alto_valor_adquisitivo)

      X = df.drop(columns=['tiene_alto_valor_adquisitivo'])
      y = label_encoder.transform(df.tiene_alto_valor_adquisitivo)

      return X, y, df, label_encoder

    return df

In [77]:
df_edit = pd.DataFrame()
df_edit = df.copy()

X, y, df_new, y_encoder = feature_engineering_xg(df_edit)
df_new.head()

,tiene_alto_valor_adquisitivo,opera_en_bolsa,es_hombre,casado_trabajo,categoria_de_trabajo_empleado_provincial,categoria_de_trabajo_monotibutista,categoria_de_trabajo_relacion_de_dependencia,categoria_de_trabajo_responsable_inscripto,categoria_de_trabajo_sin_trabajo,categoria_de_trabajo_trabajo_voluntariado,educacion_alcanzada_2_anio,educacion_alcanzada_3_anio,educacion_alcanzada_4_anio,educacion_alcanzada_5-6_grado,educacion_alcanzada_5_anio,educacion_alcanzada_7-8_grado,educacion_alcanzada_9_grado,educacion_alcanzada_preescolar,educacion_alcanzada_universiada_5_anio,educacion_alcanzada_universiada_6_anio,educacion_alcanzada_universidad_1_anio,educacion_alcanzada_universidad_2_anio,educacion_alcanzada_universidad_3_anio,educacion_alcanzada_universidad_4_anio,estado_marital_matrimonio_militar,estado_marital_pareja_no_presente,estado_marital_separado,estado_marital_sin_matrimonio,estado_marital_viudo_a,religion_cristianismo,religion_judaismo,religion_otro,rol_familiar_registrado_otro,rol_familiar_registrado_sin_familia,rol_familiar_registrado_soltero_a,trabajo_entretenimiento,trabajo_inspector,trabajo_limpiador,trabajo_otros,trabajo_profesional_especializado,trabajo_reparador,trabajo_sector_primario,trabajo_seguridad,trabajo_servicio_domestico,trabajo_soporte_tecnico,trabajo_transporte,trabajo_ventas
0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [78]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [79]:
#Instancio el xgboost
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', colsample_bytree = 0.35, learning_rate = 0.13, 
                          max_depth = 5, alpha = 5, n_estimators = 60)

In [80]:
#Entreno
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.35, gamma=0,
             importance_type='gain', learning_rate=0.13, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=60,
             n_jobs=1, nthread=None, objective='binary:logistic',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [81]:
#prediccion
preds = xg_reg.predict(X_test)

In [82]:
#Convierto las probabilidades que  devuele XGBoost para calcular las metricas
def get_int_predictions(preds):
  predictions = []
  
  for i in preds:
    if i < 0.6:
      predictions.append(0)
    else:
      predictions.append(1)

  return predictions

In [83]:
int_predictions = get_int_predictions(preds)

In [84]:
# Obtener metricas de error
print("Matriz de confusion")
print(confusion_matrix(y_test, int_predictions))
print("\n")

print("Porcentaje de aciertos")
print(round(accuracy_score(y_test, int_predictions),3))
print("\n")

print("Precision")
print(round(precision_score(y_test, int_predictions),3))
print("\n")

print("Recall")
print(round(recall_score(y_test, int_predictions),3))
print("\n")

print("F1 Score")
print(round(f1_score(y_test, int_predictions),3))

Matriz de confusion
[[4768  159]
 [ 924  662]]


Porcentaje de aciertos
0.834


Precision
0.806


Recall
0.417


F1 Score
0.55


In [85]:
#Porcentaje de acierto de las predicciones
print("AUC: %f" % (roc_auc_score(y_test, preds)))

AUC: 0.886042


In [63]:
params = {"objective":"reg:linear",'colsample_bytree': 0.35,
          'learning_rate': 0.13, 'max_depth': 5, 'alpha': 5, 'silent':1}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=60, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123);
cv_results.iloc[-1]['test-auc-mean']

0.8812865999999999

# Prediccion sobre el archivo final

In [86]:
data_test_final = pd.DataFrame()
data_test_final = pd.read_csv("/content/test.csv")

data_test_final.drop(columns = ["representatividad_poblacional"], inplace=True)
df.rename(columns={'ganancia_perdida_declarada_bolsa_argentina': 'ganancia_perdida_declarada_bolsa_argentina'}, inplace=True)
data_test_final.head()

,id,anios_estudiados,barrio,categoria_de_trabajo,edad,educacion_alcanzada,estado_marital,ganancia_perdida_declarada_bolsa_argentina,genero,horas_trabajo_registradas,religion,rol_familiar_registrado,trabajo
0,1,11,Palermo,relacion_de_dependencia,25,2_anio,sin_matrimonio,0,hombre,40,judaismo,con_hijos,inspector
1,2,13,Palermo,relacion_de_dependencia,38,4_anio,matrimonio_civil,0,hombre,50,cristianismo,casado,sector_primario
2,3,16,Palermo,empleado_municipal,28,universidad_2_anio,matrimonio_civil,0,hombre,40,cristianismo,casado,seguridad
3,4,14,Palermo,relacion_de_dependencia,44,5_anio,matrimonio_civil,7688,hombre,40,judaismo,casado,inspector
4,5,14,Palermo,NaN,18,5_anio,sin_matrimonio,0,mujer,30,cristianismo,con_hijos,NaN


In [87]:
df_test_final = feature_engineering_xg(data_test_final)
df_test_final.head()

,id,opera_en_bolsa,es_hombre,casado_trabajo,categoria_de_trabajo_empleado_provincial,categoria_de_trabajo_monotibutista,categoria_de_trabajo_relacion_de_dependencia,categoria_de_trabajo_responsable_inscripto,categoria_de_trabajo_sin_trabajo,categoria_de_trabajo_trabajo_voluntariado,educacion_alcanzada_2_anio,educacion_alcanzada_3_anio,educacion_alcanzada_4_anio,educacion_alcanzada_5-6_grado,educacion_alcanzada_5_anio,educacion_alcanzada_7-8_grado,educacion_alcanzada_9_grado,educacion_alcanzada_preescolar,educacion_alcanzada_universiada_5_anio,educacion_alcanzada_universiada_6_anio,educacion_alcanzada_universidad_1_anio,educacion_alcanzada_universidad_2_anio,educacion_alcanzada_universidad_3_anio,educacion_alcanzada_universidad_4_anio,estado_marital_matrimonio_militar,estado_marital_pareja_no_presente,estado_marital_separado,estado_marital_sin_matrimonio,estado_marital_viudo_a,religion_cristianismo,religion_judaismo,religion_otro,rol_familiar_registrado_otro,rol_familiar_registrado_sin_familia,rol_familiar_registrado_soltero_a,trabajo_entretenimiento,trabajo_inspector,trabajo_limpiador,trabajo_otros,trabajo_profesional_especializado,trabajo_reparador,trabajo_sector_primario,trabajo_seguridad,trabajo_servicio_domestico,trabajo_soporte_tecnico,trabajo_transporte,trabajo_ventas
0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
#prediccion
predictions_final = xg_reg.predict(df_test_final.iloc[:, 1:47])

In [89]:
predictions_final

array([0.01211303, 0.14720899, 0.37793046, ..., 0.6700527 , 0.7555551 ,
       0.7484571 ], dtype=float32)

In [90]:
#Armo el submit
int_predictions_final = get_int_predictions(predictions_final)

pd_predictions_final = pd.DataFrame(columns=["id", "tiene_alto_valor_adquisitivo"])

pd_predictions_final.id = df_test_final.id
pd_predictions_final.tiene_alto_valor_adquisitivo = int_predictions_final

pd_predictions_final.head()

,id,tiene_alto_valor_adquisitivo
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [91]:
out_df = pd_predictions_final[['id', 'tiene_alto_valor_adquisitivo']]
out_df.set_index('id', inplace=True)
out_df.head()

,tiene_alto_valor_adquisitivo
id,
1,0
2,0
3,0
4,0
5,0


In [92]:
out_df.to_csv('pd_predictions_final_xgboost.csv')

## Metricas sobre la prediccion final

In [46]:
data_targets_final = pd.read_csv("/content/targets.csv")
data_targets_final.iloc[:, 1].head()

0    0
1    0
2    1
3    0
4    0
Name: tiene_alto_valor_adquisitivo, dtype: int64

In [47]:
#Porcentaje de acierto de las predicciones
print("AUC: %f" % (roc_auc_score(data_targets_final.iloc[:, 1], preds)))

AUC: 0.492873


In [48]:
#Convierto las probabilidades que  devuele XGBoost para calcular las metricas
predictions = []
for i in preds:
  if i < 0.6:
    predictions.append(0)
  else:
    predictions.append(1)

In [50]:
# Obtener metricas de error
print("Matriz de confusion")
print(confusion_matrix(data_targets_final.iloc[:, 1], predictions))
print("\n")

print("Porcentaje de aciertos")
print(round(accuracy_score(data_targets_final.iloc[:, 1], predictions),3))
print("\n")

print("Precision")
print(round(precision_score(data_targets_final.iloc[:, 1], predictions),3))
print("\n")

print("Recall")
print(round(recall_score(data_targets_final.iloc[:, 1], predictions),3))
print("\n")

print("F1 Score")
print(round(f1_score(data_targets_final.iloc[:, 1], predictions),3))

Matriz de confusion
[[7084 1020]
 [7208  969]]


Porcentaje de aciertos
0.495


Precision
0.487


Recall
0.119


F1 Score
0.191


# Analisis de features de XGBoost

Importancia de cada feature

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [30, 30]
plt.show()

Cross Validation


In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.35,
          'learning_rate': 0.13, 'max_depth': 5, 'alpha': 10, 'silent': 1}

cv_scores = []

for n in range(10, 100, 5):
    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=n, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123);
    cv_scores.append(cv_results.iloc[-1]['test-auc-mean'])

In [ ]:
cv_scores

[0.8732512,
 0.8773476,
 0.8801178000000001,
 0.8820092,
 0.8832498000000001,
 0.8845659999999999,
 0.8853437999999999,
 0.8859429999999999,
 0.8863538,
 0.8868452,
 0.8872414,
 0.8874632,
 0.8877280000000001,
 0.887876,
 0.8880000000000001,
 0.888185,
 0.8883684000000001,
 0.8884748]

In [ ]:
abscisa=range(10, 100, 5)
plt.plot(abscisa, cv_scores)
plt.grid()
plt.xlabel('n_estimators')
plt.ylabel('Score')
plt.show()

Learning rate

In [ ]:
cv_scores = []

for n in range(1, 11):
    eta=n*0.05
    params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': eta, 'max_depth': 5, 'alpha': 10, 'silent': 1}
    print(eta)
    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123);
    cv_scores.append(cv_results.iloc[-1]['test-auc-mean'])

0.05
0.1
0.15000000000000002
0.2
0.25
0.30000000000000004
0.35000000000000003
0.4
0.45
0.5


In [ ]:
plt.plot([x * 0.05 for x in range(1, 11)], cv_scores)
plt.grid()
plt.xlabel('eta')
plt.ylabel('Score')
plt.show()

Max_depth

In [ ]:
cv_scores = []

for n in range(1, 11):
    params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': n, 'alpha': 10, 'silent': 1}

    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123);
    cv_scores.append(cv_results.iloc[-1]['test-auc-mean'])

In [ ]:
plt.plot([x for x in range(1, 11)], cv_scores)
plt.grid()
plt.xlabel('max_depth')
plt.ylabel('Score')
plt.show()

colsample_bytree (Valor muy alto: overfitting)

In [ ]:
cv_scores = []

for n in range(1, 16):
    cst=n*0.05
    params = {"objective":"reg:linear",'colsample_bytree': cst,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10, 'silent': 1}

    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123);
    cv_scores.append(cv_results.iloc[-1]['test-auc-mean'])

In [ ]:
plt.plot([x*0.05 for x in range(1, 16)], cv_scores)
plt.grid()
plt.xlabel('colsample_bytree')
plt.ylabel('Score')
plt.show()

Alpha

In [ ]:
cv_scores = []

for n in range(1, 16):
    params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': n, 'silent': 1}

    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123);
    cv_scores.append(cv_results.iloc[-1]['test-auc-mean'])

In [ ]:
plt.plot([x for x in range(1, 16)], cv_scores)
plt.grid()
plt.xlabel('alpha')
plt.ylabel('Score')
plt.show()